# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df = pd.read_csv("city_data.csv")
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hofn,64.2539,-15.2082,40.69,76,100,10.04,IS,1666591275
1,1,Touros,-5.1989,-35.4608,77.23,81,12,10.87,BR,1666591281
2,2,Airai,-8.9266,125.4092,76.55,52,82,2.17,TL,1666591282
3,3,Ushuaia,-54.8000,-68.3000,34.68,94,84,3.85,AR,1666591282
4,4,Punta Arenas,-53.1500,-70.9167,42.12,81,35,5.68,CL,1666591283


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = city_df[["Lat", "Lng"]]

weights = city_df["Humidity"].astype(float)

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights,
                                 dissipating=False, max_intensity=10,
                                point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
weather = city_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Cloudiness"] == 0) & \
                                 (city_df["Max Temp"] >= 70) & (city_df["Max Temp"] <= 80)]

weather.dropna
weather.head() 

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
110,110,Saint-Pierre,-21.3393,55.4781,78.91,65,0,5.75,RE,1666591216
114,114,São Filipe,14.8961,-24.4956,77.36,73,0,5.10,CV,1666591383
295,295,High Rock,26.6208,-78.2833,78.19,63,0,9.13,BS,1666591546
354,354,Port Saint Lucie,27.2939,-80.3503,72.97,78,0,3.00,US,1666591341
373,373,Sabzawār,36.2126,57.6819,70.18,34,0,2.91,IR,1666591616


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = weather.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
110,Saint-Pierre,RE,-21.3393,55.4781,
114,São Filipe,CV,14.8961,-24.4956,
295,High Rock,BS,26.6208,-78.2833,
354,Port Saint Lucie,US,27.2939,-80.3503,
373,Sabzawār,IR,36.2126,57.6819,


In [9]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest Hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing Result... Skipping.")
        
    print("-----------------------")
    
    
    
print("------End Of Search------")

Retrieving Results for Index 110: Saint-Pierre.
Closest Hotel in Saint-Pierre is Villa Delisle.
-----------------------
Retrieving Results for Index 114: São Filipe.
Closest Hotel in São Filipe is Hotel Xaguate.
-----------------------
Retrieving Results for Index 295: High Rock.
Closest Hotel in High Rock is Hideaway Bahamas Beach Club.
-----------------------
Retrieving Results for Index 354: Port Saint Lucie.
Closest Hotel in Port Saint Lucie is Best Western Port St. Lucie.
-----------------------
Retrieving Results for Index 373: Sabzawār.
Closest Hotel in Sabzawār is Sabzevar Tourism hotel.
-----------------------
Retrieving Results for Index 409: Andros Town.
Closest Hotel in Andros Town is Dream Villas.
-----------------------
Retrieving Results for Index 551: Nouakchott.
Closest Hotel in Nouakchott is Monotel (Dar El Barka).
-----------------------
------End Of Search------


In [11]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
110,Saint-Pierre,RE,-21.3393,55.4781,Villa Delisle
114,São Filipe,CV,14.8961,-24.4956,Hotel Xaguate
295,High Rock,BS,26.6208,-78.2833,Hideaway Bahamas Beach Club
354,Port Saint Lucie,US,27.2939,-80.3503,Best Western Port St. Lucie
373,Sabzawār,IR,36.2126,57.6819,Sabzevar Tourism hotel
409,Andros Town,BS,24.7000,-77.7667,Dream Villas
551,Nouakchott,MR,18.0858,-15.9785,Monotel (Dar El Barka)


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# DATE OF ANALYSIS --- 25/10/22